# Reading and Writing Parquet Files
Parquet is a high-density and performance file organization, the following section demonstrates how to with Parquet and import/export it to iguazio database

see more details:  https://arrow.apache.org/docs/python/parquet.html

In [2]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

In [6]:
# read some data into a dataframe and show its data & metadata
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header = None)

# specify columns extracted from wbdc.names
data.columns = ["id","diagnosis","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean",
                "compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean",
                "radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se",
                "concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst",
                "perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst",
                "concave points_worst","symmetry_worst","fractal_dimension_worst"] 

data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Write the data into a Parquet on iguazio file system
Note: iguazio file system is mounted under `/v3io/{data-container-name}`

In [11]:
#write the dataframe into a parquet (on iguazio file system)
filepath = '/v3io/bigdata/examples/example.parquet'
table = pa.Table.from_pandas(data)
pq.write_table(table, filepath)

In [12]:
# read the parquet into memory and print the head 
table2 = pq.read_table(filepath)
table2.to_pandas().head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Advanced usage

In [13]:
# read specific columns to save bandwidth/memory and accelerate load
pq.read_table(filepath, columns=['id', 'area_mean']).to_pandas().head()

,id,area_mean
0,842302,1001.0
1,842517,1326.0
2,84300903,1203.0
3,84348301,386.1
4,84358402,1297.0


In [25]:
# read specific columns, if previously written by Pandas (read extra metadata)
pq.read_pandas('example.parquet', columns=['age', 'job']).to_pandas().head()

""
0
1
2


In [ ]:
parquet_file = pq.ParquetFile('example.parquet')
parquet_file.schema

In [ ]:
parquet_file.metadata

In [ ]:
# write to multiple partitioned files 
pq.write_to_dataset(table, root_path='my_parq', partition_cols=['HPCP'])

In [ ]:
# read partitioned table
table3 = pq.read_table('my_parq')
table3.to_pandas().head()

In [ ]:
# read with multiple threads
pq.read_table('my_parq', use_threads=4).to_pandas().head()

## Remove Data

In [ ]:
# unmark the comment
#!rm -rf /v3io/bigdata/examples/*